<a href="https://www.kaggle.com/code/rohandwivedi2005/capstone?scriptVersionId=234965537" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Chatbot for google-genai SDK

Install Dependencies

In [1]:
pip install requests beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import time
import json
import os
import re

In [3]:
URL_1 = "https://googleapis.github.io/python-genai/"
URL_2 = "https://googleapis.github.io/python-genai/genai.html"
LIST = [URL_1,URL_2]
OUTPUT_FILE = "GENAI_SDK_DOCS.txt"
REQUEST_DELAY = 1
HEADERS = {'User-Agent': 'SimpleScraper/1.0'}


# Construct the full path for clarity
output_path = os.path.join("/kaggle/working/", OUTPUT_FILE)
print(f"Saving data to: {output_path}")
print("-" * 30)

try:
    with open(output_path, 'w', encoding='utf-8') as f:
        for current_url in LIST:
            time.sleep(REQUEST_DELAY) # Delay *before* request
            response = requests.get(current_url, headers=HEADERS, timeout=15) # Increased timeout slightly
            response.raise_for_status() # Check for HTTP errors (4xx, 5xx)
        
            soup = BeautifulSoup(response.content, 'html.parser')
            text = soup.get_text(separator=' ', strip=True)
            text = ' '.join(text.split())
            if text:
                                # --- Modified Section: Write Plain Text ---
                # 1. Write a header indicating the source URL
                f.write(f"--- URL: {current_url} ---\n") # Add a separator line with the URL

                # 2. Write the extracted text content
                f.write(text + "\n") # Write the text itself

                # 3. Add an extra newline for better separation between entries
                f.write("\n")
                # --- End Modified Section ---

                print(f"  -> Text extracted and saved for {current_url}")
                    
            else:
                print(f"  -> No significant text content found for {current_url}. Skipping.")


                
               


except requests.exceptions.RequestException as e:
    print(f"  -> Network/HTTP Error processing {current_url}: {e}")
except Exception as e:
    rint(f"  -> Error processing {current_url}: {e}")

            # Add a small break condition during development/testing if needed
            # if data_written_count > 10: break

except IOError as e:
     print(f"FATAL ERROR: Could not open or write to output file {output_path}: {e}")

print("-" * 30)
print(f"Crawling finished.")
print(f"Data saved in Kaggle environment at: {output_path}")


PATH = output_path



Saving data to: /kaggle/working/GENAI_SDK_DOCS.txt
------------------------------
  -> Text extracted and saved for https://googleapis.github.io/python-genai/
  -> Text extracted and saved for https://googleapis.github.io/python-genai/genai.html
------------------------------
Crawling finished.
Data saved in Kaggle environment at: /kaggle/working/GENAI_SDK_DOCS.txt


In [4]:
PATH = output_path

In [5]:
import os
for dirname, _, filenames in os.walk('/kaggle/working'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/working/__notebook__.ipynb
/kaggle/working/GENAI_SDK_DOCS.txt


In [6]:
!pip uninstall -qqy jupyterlab kfp  # Remove unused conflicting packages
!pip install -qU "google-genai==1.7.0" "chromadb==0.6.3"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 2.9 MB/s eta 0:00

In [7]:
from google import genai
from google.genai import types

from IPython.display import Markdown

genai.__version__

'1.7.0'

In [8]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

In [9]:

client = genai.Client(api_key=GOOGLE_API_KEY)

for m in client.models.list():
    if "embedContent" in m.supported_actions:
        print(m.name)
        

models/embedding-001
models/text-embedding-004
models/gemini-embedding-exp-03-07
models/gemini-embedding-exp


In [10]:
import chromadb
from chromadb import Documents, EmbeddingFunction, Embeddings
from google.genai import types
from google.api_core import retry

In [11]:
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

class GeminiEmbeddingFunction(EmbeddingFunction):
    # Specify whether to generate embeddings for documents, or queries
    document_mode = True
    

    @retry.Retry(predicate=is_retriable)
    def __call__(self, input: Documents) -> Embeddings:
        if self.document_mode:
            embedding_task = "retrieval_document"
        else:
            embedding_task = "retrieval_query"

        response = client.models.embed_content(
            model="models/text-embedding-004",
            contents=input,
            config=types.EmbedContentConfig(
                task_type=embedding_task,
            ),
        )
        return [e.values for e in response.embeddings]

In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [13]:
with open(PATH,"r") as f:
    text = f.read()

In [14]:
CHUNK_SIZE = 2000 # Max number of characters per chunk
CHUNK_OVERLAP = 200 # Number of characters to overlap between chunks

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    length_function=len,
    is_separator_regex=False,
)

chunks = text_splitter.split_text(text)

In [15]:
print(len(chunks))
print(len(chunks[3]))

780
1999


In [16]:
DB_NAME = "googlecardb"

embed_fn = GeminiEmbeddingFunction()
embed_fn.document_mode = True

chroma_client = chromadb.Client()
db = chroma_client.get_or_create_collection(name=DB_NAME, embedding_function=embed_fn)
current_id_offset = 0
total_chunks = len(chunks)
API_BATCH_SIZE = 100

batch_size = 100

for i in range(0, total_chunks, API_BATCH_SIZE):
    batch_chunks = chunks[i : i + API_BATCH_SIZE]
    
    batch_ids = [str(current_id_offset + j) for j in range(len(batch_chunks))]
    current_id_offset += len(batch_chunks)

    db.add(documents=batch_chunks,ids=batch_ids)
    

In [17]:
db.count()

780

In [18]:
from IPython.display import Markdown
embed_fn.document_mode = False

query = "how to import google-genai sdk"

result = db.query(query_texts=[query], n_results=5)


context = result['documents'][0]

prompt = f"""
Role : You are a teacher for the new google genai sdk whatever the questions you get you will answer the in a teaching manner


Answer the following question based ONLY on the provided context the answers:

Context:
{context}

Question:
{query}

Answer:
"""

response = client.models.generate_content(
     model='gemini-2.0-flash',
     contents=prompt,
    
)

Markdown(response.text)



Okay, let's learn how to import the Google Gen AI SDK in Python.

Based on the documentation, here's the import statement you'll use:

```python
from google import genai
from google.genai import types
```

This will import the core `genai` module and the `types` submodule, which likely contains helpful type definitions for working with the SDK.

Now you're ready to start using the SDK! The next step would be to create a client, and the documentation provides instructions for that as well, depending on whether you're using the Gemini Developer API or Vertex AI.
